In [1]:
import pandas as pd
import re
import os
import numpy as np

"""

refseq - ncbi-ftp/gene/DATA/gene2refseq.gz
pubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz
ensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz
human - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
mart - useast.ensembl.org/index.html
neighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz
orthologs - ncbi-ftp/gene/DATA/gene_orthologs.gz

"""

# refseq = pd.read_table('gene2refseq')
# pubmed = pd.read_table('gene2pubmed')
# ensembl = pd.read_table('gene2ensembl')
# mart = pd.read_csv('mart_export.txt',delimiter = ',')
# neighbors = pd.read_table('gene_neighbors')
# orthologs = pd.read_table('gene_orthologs')


'\n\nrefseq - ncbi-ftp/gene/DATA/gene2refseq.gz\npubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz\nensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz\nhuman - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz\nmart - useast.ensembl.org/index.html\nneighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz\northologs - ncbi-ftp/gene/DATA/gene_orthologs.gz\n\n'

In [2]:
human = pd.read_table('Homo_sapiens.gene_info')
aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep


In [3]:
def compare_sets(list_data):
    for ele in list_data:
        col_names = ele.columns
        print(col_names)
data_list = refseq,pubmed,ensembl,human,mart,neighbors,orthologs
# compare_sets(data_list)

NameError: name 'refseq' is not defined

In [ ]:
def combined_translate(input_file, file_type, trans_list):
    if (file_type == 'csv'):
        df = pd.read_csv(input_file)
    if (file_type == 'excel'):
        df = pd.read_excel(input_file)
    if (file_type == 'other'):
        df = pd.read_table(input_file)
    
    for item in trans_list:
        test_col = item[0]
        reference_df = item[1]
        reference_col = item[2]
        output_col = item[3]
        
        
        start_param = df[test_col].tolist()
        in_list = reference_df[reference_col].tolist()
        out_listoflist = []

        for i in range(len(start_param)):
            param = start_param[i]

            match_list=[]
            for j in range(len(in_list)):
                if str(param) in str(in_list[j]):
                    index_match = reference_df.iloc[j]
                    match_list.append(index_match)

            out_list = []
            for k in range(len(match_list)):
                if (output_col == 'MATCH'):
                    out_list.append(match_list[k][reference_col])
                else: 
                    out_list.append(match_list[k][output_col])

            out_listoflist.append(out_list)
        out_colname = 'output ' + reference_col
        df[out_colname] = out_listoflist

    
        nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
        matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
        nonmatches = df.iloc[nonmatches_ind]
        matches = df.iloc[matches_ind]
        
        return str(round(100*len(nonmatches)/len(df),4))+' % mismatch '
#         print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",input_file, 'to')
    
#     print(df)
    
        
          

In [ ]:
### Name of input data file
input_file = 'md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx'

### Type of file (excel, csv, or other)
file_type = 'excel'

### translation 1 test col name
test_col1 = 'Gene Symbol'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'MATCH'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'md_2018_05_18_zhu_md-d-17-05991_sdc1.xlsx'
file_type = 'excel'
test_col1 = 'Entrez Gene'
ref_df1 = human
ref_col1 = 'GeneID'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'RA_loci.txt'
file_type = 'csv'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'Yamamoto.txt'
file_type = 'other'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))



input_file = 'RADB.txt'
file_type = 'other'
test_col1 = 'Gene'
ref_df1 = human
ref_col1 = 'Aliases'
out_col1 = 'MATCH'
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
print(combined_translate(input_file,file_type,trans_list))


In [ ]:
def match_lists(data_list):
    for i in range(len(data_list)):
        for j in range(len(data_list)):
            if (i != j):
                ### translation 1 test col name
                test_col1 = data_list[i][1]

                ### translation 1 reference dataset (human,gene,mart)
                if (data_list[j][2] == 'csv'):
                    ref_df1 = pd.read_csv(data_list[j][0])
                if (data_list[j][2] == 'excel'):
                    ref_df1 = pd.read_excel(data_list[j][0])
                if (data_list[j][2] == 'other'):
                    ref_df1 = pd.read_table(data_list[j][0])

                ### translation 1 reference col name
                ref_col1 = data_list[j][1]

                ### translation 1 output col name
                out_col1 = 'MATCH'
                
                trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
                match_percent = combined_translate(data_list[i][0], data_list[i][2], trans_list)
                print(match_percent, data_list[i][0], 'to', data_list[j][0])
#                 print(match_percent)

### Rheumetoid Arthritis data

In [4]:
md_2018 = pd.read_excel('md_2018.xlsx')
# https://journals.lww.com/md-journal/Fulltext/2018/06010/Identification_of_key_genes_in_rheumatoid.86.aspx
# A total of 313 genes (DEGs) were identified to be differentially expressed between RA and NC samples

ra_loci = pd.read_csv('RA_loci.txt')
# https://academic.oup.com/view-large/27924154
# Genetic loci associated with susceptibility to RA

yamamoto = pd.read_table('Yamamoto.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4729856/
# RA susceptible genes

radb = pd.read_table('RADB.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4164886/
# Genes and genetic regions that have the strongest association with RA susceptibility. 

okada = pd.read_table('okada.txt')
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3944098/#SD2
# Novel rheumatoid arthritis risk loci identified by trans-ethnic GWAS meta-analysis in >100,000 subjects.


l1 = ['md_2018.xlsx', 'Gene Symbol','excel']
l2 = ['ra_loci.txt','Gene','csv']
l3 = ['Yamamoto.txt','Gene','other']
l4 = ['RADB.txt','Gene','other']
l5 = ['okada.txt','Gene','other']

match_lists([l1,l2,l3,l4,l5])


NameError: name 'match_lists' is not defined

### t2 diabetes

In [5]:
dia_supp = pd.read_excel('NIHMS795012-supplement-supp_table20.xlsx', header=2)
dia_supp


#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570412/
#SNP coverage and T2D association for 222 candidate gene regions (-10 kb/+5 kb)
gaulton_candidates = pd.read_excel('db_gaulton_candidates.xlsx')

#Stage 1 T2D SNP association for 3,531 genotyped SNPs, sorted by pSNP
gaulton = pd.read_excel('db_gaulton.xlsx')
gaulton

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4377835/
# Genetic loci associated with risk of T2D.
prasad = pd.read_table('prasad_t2d.txt')

# Genetic loci associated with glycemic traits.
prasad_gly = pd.read_table('prasad_glycemic.txt')
prasad_gly

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5537262/
t2diacod = pd.read_table('t2diacod.txt')



### heart disease

In [6]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3319439/
# Genes of interest within or near associated interval (genetic loci mapped by GWAS for myocardial 
# infarction or coronary artery disease)
kathiresan = pd.read_table('kathiresan.txt')

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5266753/
# 29 GWAS studies are included that identified more than 150 genomic loci associated with CAD and AMI 
barth = pd.read_excel('NIHMS764306-supplement-2.xlsx')
barth

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3684247/
joehanes = pd.read_table('Joehanes.txt')
joehanes

# https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/30586722/
aragam = pd.read_table('Aragam.txt')



In [7]:
def combine_datasets(listoflist):
    
    combined_data=[]
    label_list = []
    
    for i in range(len(listoflist)):
        list1 = listoflist[i]
        combined_disease=[]
        colnames = []
        for j in range(len(list1)):
            dataset = list1[j][0]
            col_name = list1[j][1]
            gene_list = dataset[col_name].tolist()
            print(len(gene_list))
        
            combined_disease.append([gene_list])
            colnames.append(col_name)
        combined_data.append(combined_disease)
    
        label = [i] * len(list1)
        label_list.append(label)
    flat_list = [item for sublist in combined_data for item in sublist]
    flat_labels = [item for sublist in label_list for item in sublist]
    
    df = pd.DataFrame(flat_list)
    df['disease_type'] = flat_labels
#     df.columns = colnames
        
    return df

ra_list = [[md_2018, 'Gene Symbol'],
[ra_loci, 'Gene'],
[yamamoto, 'Gene'],
[radb, 'Gene'],
[okada, 'Gene']]



t2d_list = [[dia_supp, 'Gene name'],
[gaulton_candidates, 'Gene symbol(s)'],
[gaulton, 'Gene symbol'],
[prasad, 'Gene/Nearest Gene'],
[prasad_gly, 'Gene/Nearest Gene'],
[t2diacod, 'Gene']]

heart_list = [[kathiresan, 'Gene'],
[barth, 'Reported Gene(s)'],
[joehanes, 'Gene Symbol'],
[aragam, 'Gene']]

gene_df = combine_datasets([ra_list,t2d_list,heart_list])
# t2d_df = combine_datasets(t2d_list)
# heart_df = combine_datasets(heart_list)



313
106
105
21
100
635
216
3531
153
24
140
49
214
35
30


In [22]:
x = gene_df.iloc[:,0].values.transpose()
y = gene_df.iloc[:,1].values.transpose()
# y = np.array([y])

for i in range(gene_df.shape[0]):
    x_train = np.delete(x, i)
    y_train = np.delete(y, i)
    print(y_train)


[0 0 0 0 1 1 1 1 1 1 2 2 2 2]
[0 0 0 0 1 1 1 1 1 1 2 2 2 2]
[0 0 0 0 1 1 1 1 1 1 2 2 2 2]
[0 0 0 0 1 1 1 1 1 1 2 2 2 2]
[0 0 0 0 1 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 1 2 2 2]
[0 0 0 0 0 1 1 1 1 1 1 2 2 2]
[0 0 0 0 0 1 1 1 1 1 1 2 2 2]
[0 0 0 0 0 1 1 1 1 1 1 2 2 2]


In [11]:
a = np.array([1,2,3,5,6])
np.delete(a, 3)

array([1, 2, 3, 6])